In [ ]:
using Base.Iterators
using Roots
using IntervalSets
using JuMP
using Ipopt
using CompEcon
using QuantEcon
using BenchmarkTools
using LinearAlgebra
using Plots
using Combinatorics

### Model settings

In [ ]:
# Number of households
N = 3;

# income shocks and their transition probabilities of the household 1
inc1 = [2/3, 4/3];
P1 = hcat(repeat([0.1 0.9], 2));

# income shocks and their transition probabilities of the household 2
inc2 = [2/3, 4/3];
P2 = hcat(repeat([0.1 0.9], 2));

# income shocks and their transition probabilities of the household 2
inc3 = [2/3, 4/3];
P3 = hcat(repeat([0.1 0.9], 2));

# transition super-matrix of income shocks
R = kron(P3, P2, P1);

# number of states
# (number of income states for HH 1 times
# number of income states for HH 2)
num_states = length(inc1) * length(inc2) * length(inc3);

# Income matrix
# (col 1: HH 1's income, col 2: HH 2's income, col 3: HH 3's income)
inc_mat = vec(collect(product(inc1, inc2, inc3)));

# Aggregate income in each state
inc_ag = sum.(inc_mat);


In [3]:
util = function(c, σ::Float64);
    if σ != 1.0 
        return (c^(1.0 - σ) - 1.0) / (1.0 - σ)
    elseif σ == 1.0
        return log(c)
    end
end;

util_prime = (c, σ::Float64) -> c^(- σ);

### Value of autarky

In [4]:
δ = 0.95 # time discount factor;
σ₁ = 1.0 # coefficient of relative risk aversion of HH1;
σ₂ = 1.0 # coefficient of relative risk aversion of HH2;
σ₃ = 1.0 # coefficient of relative risk aversion of HH3;
ϕ = 0.0 # punishment under autarky;

In [5]:
inc_vec = [inc1, inc2, inc3];
P_vec = [P1, P2, P3];
σ_vec = [σ₁, σ₂, σ₃];

In [6]:
function Uaut_func(
        inc_vec, P_vec, σ_vec,
        δ, ϕ,
        util, util_prime, N
    )
    
    Uaut_vec = [zeros(length(inc_vec[i])) for i in 1:N];
    
    for i in 1:N
        Utmp_aut = zeros(length(inc_vec[i]));
        iter = 1
        diff = 1.0;
        while (diff > 1e-12) & (iter < 1000)
            Utmp_aut_new = util.(inc_vec[i] .* (1.0 - ϕ), σ_vec[i]) .+ δ .* (P_vec[i] * Utmp_aut);
            diff = maximum(abs.(Utmp_aut_new - Utmp_aut));
            Utmp_aut = Utmp_aut_new;
            iter += 1;
        end
        Uaut_vec[i] = Utmp_aut;
    end
          
#     # Matrix of expected utilities of autarky
#     Uaut = vec(collect(product(U1_aut, U2_aut)));
    return Uaut_vec
end

@time Uaut_vec = Uaut_func(inc_vec, P_vec, σ_vec, δ, ϕ, util, util_prime, N);

Uaut_prod = vec(collect(Uaut_vec[1]));
for j in 2:N
    Uaut_prod = vec(collect(product(Uaut_prod, Uaut_vec[j])))
    Uaut_prod = [reduce(vcat, [i...]) for i in Uaut_prod]
end


  0.390047 seconds (1.72 M allocations: 108.077 MiB, 3.98% gc time, 98.80% compilation time)


### Grid of relative Pareto weights and consumption on the grid points

In [7]:
# Number of grid points on relative Pareto weight (x)
num_grid_x = 150;

# Grid points of relative Pareto weights (reference: HH 1)
xmin_vec = zeros(N - 1);
xmax_vec = zeros(N - 1);

for i in 2:N
    xmin_vec[i - 1] = util_prime(maximum(inc_vec[1]), σ_vec[1]) / util_prime(minimum(inc_vec[i] * (1.0 - ϕ)), σ_vec[i]);
    xmax_vec[i - 1] = util_prime(minimum(inc_vec[1] * (1.0 - ϕ)), σ_vec[1]) / util_prime(maximum(inc_vec[i]), σ_vec[i]);
end

x_mat = zeros(N - 1, num_grid_x);
for i in 2:N
    x_mat[i - 1, :] = exp.(range(log(xmin_vec[i - 1]), log(xmax_vec[i - 1]), length = num_grid_x));
end

x_ind_prod = vec(collect(product(1:num_grid_x)))
x_ind_prod = [[i...] for i in x_ind_prod]

if N >= 3
    for j in 2:(N - 1)
        x_ind_prod = vec(collect(product(x_ind_prod, 1:num_grid_x)))
        x_ind_prod = [reduce(vcat, [i...]) for i in x_ind_prod]
    end
end

x_grid_prod = vec(collect(product(x_mat[2 - 1, :])))
x_grid_prod = [[i...] for i in x_grid_prod]

if N >= 3
    for i in 3:N
        x_grid_prod = vec(collect(product(x_grid_prod, x_mat[i - 1, :])))
        x_grid_prod = [reduce(vcat, [i...]) for i in x_grid_prod]
    end
end

x_grid_prod = reduce(hcat, x_grid_prod);


$$
    \frac{u_1'(c_{1t})}{u_i'(c_{it})} = x_{it} \quad \forall i = 2, \dots, N.
$$

In [8]:
cons_array = zeros(N, num_states, length(x_ind_prod));

@time for s_ind in 1:num_states, x_ind in 1:length(x_ind_prod)
    x_ind_vec = x_ind_prod[x_ind];

    model = Model(Ipopt.Optimizer);
    set_silent(model)

    @variable(model, c[i = 1:N] >= 1e-6);

    for i in 1:N
        register(model, Symbol("util_", i), 1, x -> util(x, σ_vec[i]), x -> util_prime(x, σ_vec[i]); autodiff = true)
        register(model, Symbol("util_prime_", i), 1, x -> util_prime(x, σ_vec[i]); autodiff = true)
    end

    @NLobjective(model, Max, 0);
    for i in 2:N
        add_nonlinear_constraint(
            model,
            :(util_prime_1($(c[1])) / $(Symbol("util_prime_", i))($(c[i])) == $(x_mat[i - 1, x_ind_vec[i - 1]]));
        )
    end;
    @NLconstraint(model, sum(c[i] for i in 1:N) == inc_ag[s_ind]);

    optimize!(model);

    # print(value.(c), "\n")
    # @show value.(x)

    cons_array[:, s_ind, x_ind] = value.(c);
end;


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

524.213137 seconds (406.07 M allocations: 20.186 GiB, 3.54% gc time, 0.17% compilation time)


### Values under risk-sharing (full)

In [9]:
function V_full_func(
        inc_vec, P_vec, σ_vec,
        num_grid_x, R, num_states,
        inc_ag, xmin_vec, xmax_vec,
        x_mat, x_ind_prod, 
        δ, ϕ, util, util_prime,
        Uaut_vec, cons_array
    )
    
    # Value function iterations
    # Initial guess is expected utilities under autarky
    
    iter = 1;
    diff = 1.0;

    V_mat = (1.0 / (1.0 - δ)) .* cons_array;
    Vnew_mat = similar(V_mat);

    while (diff > 1e-8) & (iter < 500)

        for i in 1:N
            Vnew_mat[i, :, :] = util.(cons_array[i, :, :], σ_vec[i]) .+ δ .* R * V_mat[i, :, :];
        end
        diff = maximum(abs.(V_mat - Vnew_mat));
        V_mat = Vnew_mat[:, :, :];
        iter += 1;
    end    
    
    println("Took $iter iterations.")
    return(V_mat)
end

V_full_func (generic function with 1 method)

In [10]:
@time V_full = V_full_func(
        inc_vec, P_vec, σ_vec,
        num_grid_x, R, num_states,
        inc_ag, xmin_vec, xmax_vec,
        x_mat, x_ind_prod, 
        δ, ϕ, util, util_prime,
        Uaut_vec, cons_array
    );

Took 368 iterations.
  5.917652 seconds (4.14 M allocations: 10.564 GiB, 3.35% gc time, 21.87% compilation time)


### Values under risk-sharing (limited commitment)

Given the guessed value function, $V_i(s, x_t)$, proceed the following steps:

1. Calculate the value of $i$ without any participation constraints (i.e., the value function in the full risk-sharing where the relative Pareto weights do not change):
    $$V_i^{\text{tmp}}(s, x_t) = u_i(c(s, x_t)) + \delta E[V_i(s', x_t)|s].$$
2. Compare $V_i^{\text{tmp}}(s, x_t)$ and $U_i^{\text{aut}}(s)$ and consider the following cases:
    1. If $V_i^{\text{tmp}}(s, x_t) \ge U_i^{\text{aut}}(s)$ for all $i \in \{1, \dots, N\}$, then nobody's participation constraint is binding, so update the value function by $V_i^{\text{tmp}}(s, x_t)$ for everyone.
    2. If $V_i^{\text{tmp}}(s, x_t) < U_i^{\text{aut}}(s)$ for some $i \in \{2, \dots, N\}$, let S be the set of $i$'s with $V_i^{\text{tmp}}(s, x_t) < U_i^{\text{aut}}(s)$. Further consider the following cases:
        1. If $V_1^{\text{tmp}}(s, x_t) \ge U_1^{\text{aut}}(s)$, then derive $x$ such that $u_i(c(s, x)) + \delta E[V_i(s', x)|s] = U_i^{\text{aut}}(s)$ for $i \in S$ with $x^j = x^j_t$ for $j \notin S$;
        2. If $V_1^{\text{tmp}}(s, x_t) < U_1^{\text{aut}}(s)$ and $|S| = N - 2$, then derive $x$ such that $u_i(c(s, x)) + \delta E[V_i(s', x)|s] = U_i^{\text{aut}}(s)$ for $i \in \{1\} \cup S$;
        3. If $V_1^{\text{tmp}}(s, x_t) < U_1^{\text{aut}}(s)$ and $|S| < N - 2$, then derive $x$ such that $u_i(c(s, x)) + \delta E[V_i(s', x)|s] = U_i^{\text{aut}}(s)$ for $i \in \{1\} \cup S$ and $\frac{x^i_t}{x^i} = \frac{x^j_t}{x^j}$ for all pairs $i, j \notin S$.
    3. If $V_i^{\text{tmp}}(s, x_t) \ge U_i^{\text{aut}}(s)$ for all $i \in \{2, \dots, N\}$ and $V_1^{\text{tmp}}(s, x_t) < U_1^{\text{aut}}(s)$, then derive $x$ such that $u_1(c(s, x)) + \delta E[V_1(s', x)|s] = U_1^{\text{aut}}(s)$ and $\frac{x^i_t}{x^i} = \frac{x^j_t}{x^j}$ for all pairs $i, j \in \{2, \dots, N\}$.

In [11]:

# for x_ind in 1:length(x_ind_prod)
function func_tmp!(V_new, V)

    PC_violation = zeros(Int64, N)

    for s_ind in 1:num_states, x_ind in 1:length(x_ind_prod)

        for i in 1:N
            PC_violation[i] = util(cons_array[i, s_ind, x_ind], σ_vec[i]) .+ δ * R[s_ind, :]' * V[i, :, x_ind] < Uaut_prod[s_ind][i]
        end

        HH1_PC_violation = PC_violation[1];
        HHn_PC_violation = @view PC_violation[2:N]; 

        # Cases B.a
        if any(HHn_PC_violation .== 1) & (HH1_PC_violation == 0)
            
            abs_dev = zeros(length(x_ind_prod));
            for i in findall(HHn_PC_violation .== 1)
                abs_dev_value_aut = abs.(
                    util.(cons_array[i + 1, s_ind, :], σ_vec[i + 1]) .+ δ .* V[i + 1, :, :]' * R[s_ind, :] .- 
                    Uaut_prod[s_ind][i + 1]
                )
                abs_dev += abs_dev_value_aut .+ ((abs_dev_value_aut .< 0) .* 1e+5)
            end

            xₛ = [x_mat[k - 1, x_ind_prod[x_ind][k - 1]] for k in 2:N];
            for k in findall(HHn_PC_violation .== 0);
                abs_dev += abs.(xₛ[k] .- x_grid_prod[k, :]);
            end

            x_ind_new = findmin(abs_dev)[2];

            for i in 1:N
                V_new[i, s_ind, x_ind] = util(cons_array[i, s_ind, x_ind_new], σ_vec[i]) .+ δ * V[i, :, x_ind_new]' * R[s_ind, :]
            end
            
#             # Case B.a            
#             if sum(HHn_PC_violation .== 1) == N - 2
#                 abs_dev = zeros(length(x_ind_prod));
#                 for i in findall(HHn_PC_violation .== 1)
#                     abs_dev_value_aut = abs.(
#                         util.(cons_array[i + 1, s_ind, :], σ_vec[i + 1]) .+ δ .* V[i + 1, :, :]' * R[s_ind, :] .- 
#                         Uaut_prod[s_ind][i + 1]
#                     )
#                     abs_dev += abs_dev_value_aut .+ ((abs_dev_value_aut .< 0) .* 1e+5)
#                 end

#                 x_ind_new = findmin(abs_dev)[2]

#                 for i in 1:N
#                     V_new[i, s_ind, x_ind] = util(cons_array[i, s_ind, x_ind_new], σ_vec[i]) .+ δ * V[i, :, x_ind_new]' * R[s_ind, :]
#                 end
#             # Case B.b
#             elseif sum(HHn_PC_violation .== 1) < N - 2
#                 abs_dev = zeros(length(x_ind_prod));
#                 for i in findall(HHn_PC_violation .== 1)
#                     abs_dev_value_aut = abs.(
#                         util.(cons_array[i + 1, s_ind, :], σ_vec[i + 1]) .+ δ .* V[i + 1, :, :]' * R[s_ind, :] .- 
#                         Uaut_prod[s_ind][i + 1]
#                     )
#                     abs_dev += abs_dev_value_aut .+ ((abs_dev_value_aut .< 0) .* 1e+5)
#                 end

#                 xₛ = [x_mat[k - 1, x_ind_prod[x_ind][k - 1]] for k in 2:N];

#                 x_ratio_mat = zeros(length(x_ind_prod), sum(HHn_PC_violation .== 0));
#                 for k in findall(HHn_PC_violation .== 0);
#                     x_ratio_mat[:, k] = xₛ[k] ./ x_grid_prod[k, :];
#                 end

#                 x_ratio_diff_mat = zeros(length(x_ind_prod), sum(HHn_PC_violation .== 0) - 1);
#                 for k in 2:sum(HHn_PC_violation .== 0)
#                     x_ratio_diff_mat[:, k - 1] = abs.(x_ratio_mat[:, k] .- x_ratio_mat[:, 1]);
#                 end

#                 abs_dev += vec(sum(x_ratio_diff_mat, dims = 2));

#                 x_ind_new = findmin(abs_dev)[2];

#                 for i in 1:N
#                     V_new[i, s_ind, x_ind] = util(cons_array[i, s_ind, x_ind_new], σ_vec[i]) .+ δ * V[i, :, x_ind_new]' * R[s_ind, :]
#                 end
#             end
        # Cases B.b and B.c
        elseif any(HHn_PC_violation .== 1) & (HH1_PC_violation == 1)
            # Case B.c
            if sum(HHn_PC_violation .== 1) == N - 2
                abs_dev = zeros(length(x_ind_prod));

                abs_dev_value_aut = abs.(
                    util.(cons_array[1, s_ind, :], σ_vec[1]) .+ δ .* V[1, :, :]' * R[s_ind, :] .- 
                    Uaut_prod[s_ind][1]
                )
                abs_dev += abs_dev_value_aut .+ ((abs_dev_value_aut .< 0) .* 1e+5)

                for i in findall(HHn_PC_violation .== 1)
                    abs_dev_value_aut = abs.(
                        util.(cons_array[i + 1, s_ind, :], σ_vec[i + 1]) .+ δ .* V[i + 1, :, :]' * R[s_ind, :] .- 
                        Uaut_prod[s_ind][i + 1]
                    )
                    abs_dev += abs_dev_value_aut .+ ((abs_dev_value_aut .< 0) .* 1e+5)
                end

                x_ind_new = findmin(abs_dev)[2];

                for i in 1:N
                    V_new[i, s_ind, x_ind] = util(cons_array[i, s_ind, x_ind_new], σ_vec[i]) .+ δ * V[i, :, x_ind_new]' * R[s_ind, :]
                end 
            # Case B.b
            elseif sum(HHn_PC_violation .== 1) < N - 2
                abs_dev = zeros(length(x_ind_prod));

                abs_dev_value_aut = abs.(
                    util.(cons_array[1, s_ind, :], σ_vec[1]) .+ δ .* V[1, :, :]' * R[s_ind, :] .- 
                    Uaut_prod[s_ind][1]
                )
                abs_dev += abs_dev_value_aut .+ ((abs_dev_value_aut .< 0) .* 1e+5)

                for i in findall(HHn_PC_violation .== 1)
                    abs_dev_value_aut = abs.(
                        util.(cons_array[i + 1, s_ind, :], σ_vec[i + 1]) .+ δ .* V[i + 1, :, :]' * R[s_ind, :] .- 
                        Uaut_prod[s_ind][i + 1]
                    )
                    abs_dev += abs_dev_value_aut .+ ((abs_dev_value_aut .< 0) .* 1e+5)
                end

                xₛ = [x_mat[k - 1, x_ind_prod[x_ind][k - 1]] for k in 2:N];

                x_ratio_mat = zeros(length(x_ind_prod), sum(HHn_PC_violation .== 0));
                for k in findall(HHn_PC_violation .== 0);
                    x_ratio_mat[:, k] = xₛ[k] ./ x_grid_prod[k, :];
                end

                x_ratio_diff_mat = zeros(length(x_ind_prod), sum(HHn_PC_violation .== 0) - 1);
                for k in 2:sum(HHn_PC_violation .== 0)
                    x_ratio_diff_mat[:, k - 1] = abs.(x_ratio_mat[:, k] .- x_ratio_mat[:, 1]);
                end

                abs_dev += vec(sum(x_ratio_diff_mat, dims = 2));

                x_ind_new = findmin(abs_dev)[2]

                for i in 1:N
                    V_new[i, s_ind, x_ind] = util(cons_array[i, s_ind, x_ind_new], σ_vec[i]) .+ δ * V[i, :, x_ind_new]' * R[s_ind, :]
                end 
            end
        # Case C
        elseif all(HHn_PC_violation .== 0) & (HH1_PC_violation == 1)

            abs_dev = zeros(length(x_ind_prod));

            abs_dev_value_aut = abs.(
                util.(cons_array[1, s_ind, :], σ_vec[1]) .+ δ .* V[1, :, :]' * R[s_ind, :] .- 
                Uaut_prod[s_ind][1]
            )
            abs_dev += abs_dev_value_aut .+ ((abs_dev_value_aut .< 0) .* 1e+5)

            if N >= 3        
                xₛ = [x_mat[k - 1, x_ind_prod[x_ind][k - 1]] for k in 2:N];

                x_ratio_mat = zeros(length(x_ind_prod), sum(HHn_PC_violation .== 0));
                for k in findall(HHn_PC_violation .== 0);
                    x_ratio_mat[:, k] = xₛ[k] ./ x_grid_prod[k, :];
                end

                x_ratio_diff_mat = zeros(length(x_ind_prod), sum(HHn_PC_violation .== 0) - 1);
                for k in 2:sum(HHn_PC_violation .== 0)
                    x_ratio_diff_mat[:, k - 1] = abs.(x_ratio_mat[:, k] .- x_ratio_mat[:, 1]);
                end
                abs_dev += vec(sum(x_ratio_diff_mat, dims = 2));
            end

            x_ind_new = findmin(abs_dev)[2]

            for i in 1:N
                V_new[i, s_ind, x_ind] = util(cons_array[i, s_ind, x_ind_new], σ_vec[i]) .+ δ * V[i, :, x_ind_new]' * R[s_ind, :]
            end 
        # Case A
        elseif all(HHn_PC_violation .== 0) & (HH1_PC_violation == 0)
            for i in 1:N
                V_new[i, s_ind, x_ind] = util(cons_array[i, s_ind, x_ind], σ_vec[i]) .+ δ * V[i, :, x_ind]' * R[s_ind, :]
            end 
        end
    end
    
end


func_tmp! (generic function with 1 method)

In [12]:
V = V_full;
V_new = similar(V);

In [13]:
V_new

3×8×22500 Array{Float64, 3}:
[:, :, 1] =
 12.1073  12.6181  12.6181  12.3483  12.3483  12.7537  12.3013  12.5244
 12.395   12.395   12.8004  12.3483  12.5714  12.0136  12.5244  12.5244
 12.395   12.6181  12.0606  12.5714  12.5714  12.3013  12.3013  12.7067

[:, :, 2] =
 11.9662  12.4771  12.4771  12.2063  12.2063  12.6117  12.1583  12.3814
 12.2539  12.2539  12.6594  12.2063  12.4294  11.8706  12.3814  12.3814
 12.2539  12.4771  11.9186  12.4294  12.4294  12.1583  12.1583  12.5637

[:, :, 3] =
 11.8223  12.3331  12.3331  12.0613  12.0613  12.4667  12.0123  12.2354
 12.1099  12.1099  12.5154  12.0613  12.2844  11.7246  12.2354  12.2354
 12.1099  12.3331  11.7736  12.2844  12.2844  12.0123  12.0123  12.4177

...

[:, :, 22498] =
 -4.44703  -3.93621  -3.93621  -4.01408  …  -3.60862  -3.86911  -3.64597
 -4.15935  -4.15935  -3.75389  -4.01408     -4.15679  -3.64597  -3.64597
 -4.15935  -3.93621  -4.30177  -3.79094     -3.86911  -3.86911  -3.46365

[:, :, 22499] =
 -4.01212  -3.5013   -3.501

In [14]:
@time func_tmp!(V_new, V)

210.295945 seconds (56.08 M allocations: 1.063 TiB, 3.94% gc time, 0.63% compilation time)


In [15]:
V_new

3×8×22500 Array{Float64, 3}:
[:, :, 1] =
 4.55186  5.32553  4.71476  5.30423  4.71476  5.30423  4.69401  5.245
 3.71452  3.74405  4.43564  4.46691  3.69133  3.72264  4.41489  4.40766
 3.71452  3.74405  3.69133  3.72264  4.43564  4.46691  4.41489  4.40766

[:, :, 2] =
 4.55186  5.32553  4.71476  5.30423  4.71476  5.30423  4.69401  5.245
 3.71452  3.74405  4.43564  4.46691  3.69133  3.72264  4.41489  4.40766
 3.71452  3.74405  3.69133  3.72264  4.43564  4.46691  4.41489  4.40766

[:, :, 3] =
 4.55186  5.32553  4.71476  5.30423  4.71476  5.30423  4.69401  5.245
 3.71452  3.74405  4.43564  4.46691  3.69133  3.72264  4.41489  4.40766
 3.71452  3.74405  3.69133  3.72264  4.43564  4.46691  4.41489  4.40766

...

[:, :, 22498] =
 3.74736  4.4703   3.72251  4.44552  3.72251  4.44552  3.7558   4.44051
 3.8404   4.0051   4.3738   4.35219  4.3738   4.35219  4.59321  4.53355
 4.39864  4.37726  4.74597  4.7255   4.74597  4.7255   5.15148  5.09179

[:, :, 22499] =
 3.74852  4.40819  3.7231   4.44552 

In [ ]:
# s_ind = 1;

# x_next_ind = ones(Int64, N - 1) * collect(1:num_grid_x)';
# x_low_ind_vec = zeros(N - 1, num_states);
# x_high_ind_vec = zeros(N - 1, num_states);

# for i in 2:N

#     Vi_aut_diff = abs.(util.(cons_array[i, s_ind, :], σ_vec[i]) .+ δ .* (V[i, :, :]' * R[s_ind, :]) .- Uaut_prod[s_ind][i])
#     @time x_low_ind_vec[i - 1, s_ind] = findmin(reduce(hcat, x_ind_prod[findall(Vi_aut_diff .== minimum(Vi_aut_diff))])[:, min(i - 1, 1)])[1];
    
#     V1_aut_diff = abs.(util.(cons_array[1, s_ind, :], σ_vec[1]) .+ δ .* (V[1, :, :]' * R[s_ind, :]) .- Uaut_prod[s_ind][1])
#     @time x_high_ind_vec[i - 1, s_ind] = findmax(reduce(hcat, x_ind_prod[findall(V1_aut_diff .== minimum(V1_aut_diff))])[:, min(i - 1, 1)])[1];;

# #     @time x_low_ind_vec[i - 1, s_ind] = x_ind_prod[
# #         findmin(abs.(util.(cons_array[i, s_ind, :], σ_vec[i]) .+ δ .* (V[i, :, :]' * R[s_ind, :]) .- Uaut_prod[s_ind][i]))[2]
# #     ][i - 1];
# #     @time x_high_ind_vec[i - 1, s_ind] = x_ind_prod[
# #         findmin(abs.(util.(cons_array[1, s_ind, :], σ_vec[1]) .+ δ .* (V[1, :, :]' * R[s_ind, :]) .- Uaut_prod[s_ind][1]))[2]
# #     ][i - 1];
        
#     x_next_ind[i - 1, x_next_ind[i - 1, :] .<= x_low_ind_vec[i - 1, s_ind]] .= x_low_ind_vec[i - 1, s_ind];
#     x_next_ind[i - 1, x_next_ind[i - 1, :] .>= x_high_ind_vec[i - 1, s_ind]] .= x_high_ind_vec[i - 1, s_ind];    
# end




In [ ]:
# V1 = V1_full;
# V2 = V2_full;
# V1_new = zeros(num_states, num_grid_x);
# V2_new = zeros(num_states, num_grid_x);

In [ ]:
# diff = 1.0
# iter = 1
# delta_diff = 1.0
# @time while (diff > 1e-6) & (iter < 1000) & (delta_diff != 0.0)

#     for s_ind in 1:num_states
#         x_low_ind  = findmin(abs.(util.(cons1[s_ind, :], σ₁) .+ δ .* (V1' * R[s_ind, :]) .- Uaut_1[s_ind]))[2];
#         x_high_ind = findmin(abs.(util.(cons2[s_ind, :], σ₂) .+ δ .* (V2' * R[s_ind, :]) .- Uaut_2[s_ind]))[2];

#         x_next_ind = collect(1:num_grid_x);
#         x_next_ind[x_next_ind .<= x_low_ind] .= x_low_ind;
#         x_next_ind[x_next_ind .>= x_high_ind] .= x_high_ind;

#         V1_new[s_ind, :] = util.(cons1[s_ind, x_next_ind], σ₁) .+ δ .* (V1[:, x_next_ind]' * R[s_ind, :]);
#         V2_new[s_ind, :] = util.(cons2[s_ind, x_next_ind], σ₂) .+ δ .* (V2[:, x_next_ind]' * R[s_ind, :]);
#     end;

#     diff_new = maximum(abs.(V1 .- V1_new) + abs.(V2 .- V2_new));
#     delta_diff = diff - diff_new;
#     diff = diff_new

#     V1 = V1_new[:, :];
#     V2 = V2_new[:, :];
    
#     iter += 1
# end

In [ ]:
# s_ind = 1;
# x_ind = 1;

# model = Model(Ipopt.Optimizer);
# set_silent(model)

# @variable(model, c[i = 1:2] >= 1e-6);
# @variable(model, x[i = 1:num_states] >= 0.0);

# register(model, :util_1, 1, x -> util(x, σ₁), x -> util_prime(x, σ₁); autodiff = true)
# register(model, :util_2, 1, x -> util(x, σ₂), x -> util_prime(x, σ₂); autodiff = true)

# register(
#     model,
#     :V1_colloc,
#     1,
#     x -> R[s_ind, :]' * [funeval(V1_collocation[i][1], basis, [x])[1][1, 1] for i in 1:num_states];
#     autodiff = true
#     );
# register(
#     model,
#     :V1_colloc,
#     1,
#     x -> R[s_ind, :]' * [funeval(V2_collocation[i][1], basis, [x])[1][1, 1] for i in 1:num_states];
#     autodiff = true
#     );

# @NLobjective(
#     model, 
#     Max, 
#     util_1(c[1]) + δ * exp_V1_colloc
# );

# @NLconstraint(
#     model, 
#     PC1,
#     util_1(c[1]) + δ * exp_V1_colloc >= Uaut_1[s_ind]
# );
# @NLconstraint(
#     model, 
#     PC2,
#     util_2(c[2]) + δ * exp_V2_colloc >= Uaut_2[s_ind]
# );

# @NLconstraint(
#     model, 
#     sum(c[i] for i in 1:2) <= inc_ag[s_ind]
# );

# @time optimize!(model)

# print(value.(c), "\n")
# @show value.(x)



In [ ]:
# s_ind = 1;
# x_ind = 1;

# basis = fundefn(:spli, num_grid_x, xmin, xmax);
# V1_collocation = [funfitxy(basis, funnode(basis)[1], V1[i, :]) for i in 1:num_states];
# V2_collocation = [funfitxy(basis, funnode(basis)[1], V2[i, :]) for i in 1:num_states];

# x_sol = zeros(num_states, num_grid_x);

# @time for s_ind in 1:num_states, x_ind in 1:num_grid_x
#     model = Model(Ipopt.Optimizer);
#     set_silent(model)

#     @variable(model, c[i = 1:2] >= 1e-10);
#     @variable(model, x >= 0.0);

#     register(model, :util_1, 1, x -> util(x, σ₁), x -> util_prime(x, σ₁); autodiff = true)
#     register(model, :util_2, 1, x -> util(x, σ₂), x -> util_prime(x, σ₂); autodiff = true)

#     register(
#         model,
#         :V1_colloc,
#         1,
#         x -> R[s_ind, :]' * collect([funeval(V1_collocation[i][1], basis, [x])[1][1,1] for i in 1:num_states]);
#         autodiff = true
#         );
#     register(
#         model,
#         :V2_colloc,
#         1,
#         x -> R[s_ind, :]' * collect([funeval(V2_collocation[i][1], basis, [x])[1][1,1] for i in 1:num_states]);
#         autodiff = true
#         );

#     @NLobjective(
#         model, 
#         Max, 
#         util_1(c[1]) + δ * V1_colloc(x)
#     );

#     @NLconstraint(
#         model, 
#         util_1(c[1]) + δ * V1_colloc(x) >= Uaut_1[s_ind]
#     );
#     @NLconstraint(
#         model, 
#         util_2(c[2]) + δ * V2_colloc(x) >= Uaut_2[s_ind]
#     );

#     @NLconstraint(
#         model, 
#         sum(c[i] for i in 1:2) <= inc_ag[s_ind]
#     );

#     optimize!(model)
    
#     print(value.(c), "\n")
#     x_sol[s_ind, x_ind] = value(x)
# end